In [48]:
import requests
from bs4 import BeautifulSoup
import re
import json
from tqdm import tqdm_notebook as tqdm
import psycopg2  
from peewee import *
from playhouse.postgres_ext import PostgresqlExtDatabase, JSONField, ArrayField

In [49]:
with open("films","r") as f:
    fms = json.load(f)

**Ganre**

In [50]:
with open("gange","r") as f:
    ganre = json.load(f)

In [55]:
dataset=[]
for f in tqdm(fms):
    f["ganre"]=[]
    if "Жанр" not in f["info"]:
        continue
        
    for g in f["info"]["Жанр"].replace(" ","").replace(".","").split(","):
        if g!="":
            f["ganre"].append(ganre.index(g))
    
    dataset.append({
        "discr":f["name"]+" "+f["discr"],
        "ganre":f["ganre"]
    })

In [56]:
import re

class Porter:
	PERFECTIVEGROUND =  re.compile(u"((ив|ивши|ившись|ыв|ывши|ывшись)|((?<=[ая])(в|вши|вшись)))$")
	REFLEXIVE = re.compile(u"(с[яь])$")
	ADJECTIVE = re.compile(u"(ее|ие|ые|ое|ими|ыми|ей|ий|ый|ой|ем|им|ым|ом|его|ого|ему|ому|их|ых|ую|юю|ая|яя|ою|ею)$")
	PARTICIPLE = re.compile(u"((ивш|ывш|ующ)|((?<=[ая])(ем|нн|вш|ющ|щ)))$")
	VERB = re.compile(u"((ила|ыла|ена|ейте|уйте|ите|или|ыли|ей|уй|ил|ыл|им|ым|ен|ило|ыло|ено|ят|ует|уют|ит|ыт|ены|ить|ыть|ишь|ую|ю)|((?<=[ая])(ла|на|ете|йте|ли|й|л|ем|н|ло|но|ет|ют|ны|ть|ешь|нно)))$")
	NOUN = re.compile(u"(а|ев|ов|ие|ье|е|иями|ями|ами|еи|ии|и|ией|ей|ой|ий|й|иям|ям|ием|ем|ам|ом|о|у|ах|иях|ях|ы|ь|ию|ью|ю|ия|ья|я)$")
	RVRE = re.compile(u"^(.*?[аеиоуыэюя])(.*)$")
	DERIVATIONAL = re.compile(u".*[^аеиоуыэюя]+[аеиоуыэюя].*ость?$")
	DER = re.compile(u"ость?$")
	SUPERLATIVE = re.compile(u"(ейше|ейш)$")
	I = re.compile(u"и$")
	P = re.compile(u"ь$")
	NN = re.compile(u"нн$")

	def stem(word):
		word = word.lower()
		word = word.replace(u'ё', u'е')
		m = re.match(Porter.RVRE, word)
		if m.groups():
			pre = m.group(1)
			rv = m.group(2)
			temp = Porter.PERFECTIVEGROUND.sub('', rv, 1)
			if temp == rv:
				rv = Porter.REFLEXIVE.sub('', rv, 1)
				temp = Porter.ADJECTIVE.sub('', rv, 1)
				if temp != rv:
					rv = temp
					rv = Porter.PARTICIPLE.sub('', rv, 1)
				else:
					temp = Porter.VERB.sub('', rv, 1)
					if temp == rv:
						rv = Porter.NOUN.sub('', rv, 1)
					else:
						rv = temp
			else:
				rv = temp
			
			rv = Porter.I.sub('', rv, 1)

			if re.match(Porter.DERIVATIONAL, rv):
				rv = Porter.DER.sub('', rv, 1)

			temp = Porter.P.sub('', rv, 1)
			if temp == rv:
				rv = Porter.SUPERLATIVE.sub('', rv, 1)
				rv = Porter.NN.sub(u'н', rv, 1)
			else:
				rv = temp
			word = pre+rv
		return word
	stem=staticmethod(stem)

if __name__ == '__main__':
	print(Porter.stem('устойчивость'))

устойчив


In [57]:
def text_preproc(text):
    reg = re.compile('[^а-яА-Я ]')
    text = (reg.sub('', text))
    text=re.compile("\s+").sub(" ",text).lower()
    
    res=[]
    for w in text.split(" "):
        
        if len(w)>4:
            try:
                res.append(Porter.stem(w))
            except:
                print("error",w)
        else:
            res.append(w)
    
    return res



In [58]:
for d in dataset:
    res=[0 for i in range(len(ganre))]
    for g in d["ganre"]:
        res[g]=1
    d["ganre"]=res

In [59]:
vocab=[]
for d in dataset:
    vocab+=text_preproc(d["discr"])

error вссср
error ссссссс
error пгбннм
error гдплжн
error вссср
error нчтнм
error дтльн
error тщтльн
error нвзмжн
error лчйнть
error лчйнть
error зшткй
error вцспс
error члвкм
error кмнвтм
error знкмтв
error взмжн
error твлчь
error вссср
error прдкв


In [60]:
len(vocab)

1241718

In [61]:
vocab = list(set(vocab))
word2int={}
for i , v in enumerate(vocab):
    word2int[v]=i

In [62]:
len(vocab)

69127

In [63]:
dataset[0]["discr"]

'Загадочное убийство (2019)  Чтобы оживить семейные отношения полицейский из Нью-Йорка вместе со своей женой отправляются в путешествие по Европе. В самолете они знакомятся с богатым пожилым мужчиной, который приглашает их на свою роскошную яхту. Но там его убивают, а они становятся главными подозреваемыми...'

In [64]:
for d in tqdm(dataset):
    res=[]
    for w in text_preproc(d["discr"]):
        if(w!=""):
            res.append(word2int[w])
        
    d["discr"]=res

error вссср
error ссссссс
error пгбннм
error гдплжн
error вссср
error нчтнм
error дтльн
error тщтльн
error нвзмжн
error лчйнть
error лчйнть
error зшткй
error вцспс
error члвкм
error кмнвтм
error знкмтв
error взмжн
error твлчь
error вссср
error прдкв



In [65]:
dataset[0]

{'discr': [18491,
  37416,
  47724,
  25859,
  22353,
  48848,
  32806,
  34094,
  26506,
  63360,
  61886,
  29287,
  63137,
  60413,
  31703,
  35073,
  15740,
  51069,
  31703,
  27309,
  64199,
  17940,
  38429,
  51812,
  55204,
  16682,
  43865,
  57269,
  2394,
  19759,
  45521,
  27917,
  40176,
  33954,
  51436,
  58349,
  36949,
  67038,
  64199,
  27660,
  6045,
  37980],
 'ganre': [1,
  1,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0]}

In [14]:
with open("data_discr_ganre_arr","r") as f:
    dataset = json.load(f)

In [44]:
with open("vocab","w") as f:
    json.dump(word2int,f)

In [66]:
with open("data_discr_ganre_arr_word2int","w") as f:
    json.dump(dataset,f)